In [1]:
from tqdm import tqdm
import matplotlib.pyplot as plt # type: module
import matplotlib.ticker as ticker
from matplotlib import colormaps
from matplotlib.colors import Normalize

import numpy as np
import os, glob
import time
import warnings

from rur.fortranfile import FortranFile
from rur import uri, uhmi, painter, drawer
from rur.sci.photometry import measure_luminosity
from rur.sci.geometry import get_angles, euler_angle
from rur.utool import rotate_data
from scipy.ndimage import gaussian_filter
uri.timer.verbose=1
# from rur.sci.kinematics import f_getpot

from icl_IO import mode2repo, pklsave, pklload
from icl_tool import *
from icl_numba import large_isin, large_isind, isin
from icl_draw import drawsnap, add_scalebar, addtext, MakeSub_nolabel, label_to_in, fancy_axis, circle
import argparse, subprocess
from importlib import reload
import cmasher as cmr
from copy import deepcopy
from multiprocessing import Pool, shared_memory

mode = 'nh'
iout = 1026
repo, rurmode, dp = mode2repo(mode)
snap = uri.RamsesSnapshot(repo, iout, mode=rurmode)
snaps = uri.TimeSeries(snap)
snaps.read_iout_avail()
nout = snaps.iout_avail['iout']
gals = uhmi.HaloMaker.load(snap, galaxy=True, double_precision=dp)
hals = uhmi.HaloMaker.load(snap, galaxy=False, double_precision=dp)
database = f"/home/jeon/MissingSat/database"

from common_func import *

[Output 01026] Age (Gyr) : 11.624 / 13.741, z = 0.17149 (a = 0.8536)


In [2]:
LG = pklload(f"{database}/LG")
allsubs = None
states = None
for key in LG.keys():
    subs = LG[key]['subs']
    real = LG[key]['real']
    dink = real[real['state']=='dink']['hid']
    ind = isin(subs['id'], dink)
    subs['dink'][ind] = True
    subs['dink'][~ind] = False
    state = np.zeros(len(subs), dtype='<U7')
    state[ind] = 'dink'
    state[~ind] = 'pair'
    upair = real[real['state']=='upair']['hid']
    ind = isin(subs['id'], upair)
    state[ind] = 'upair'

    allsubs = subs if allsubs is None else np.hstack((allsubs, subs))
    states = state if states is None else np.hstack((states, state))
argsort = np.argsort(allsubs['id'])
allsubs = allsubs[argsort]

In [3]:
tree = pklload(f"{database}/02_main_progenitors.pickle")
tree.shape, tree.dtype, tree[0]

((3281052,),
 dtype([('lastid', '<i2'), ('timestep', '<i2'), ('id', '<i2'), ('give_score', '<f8'), ('take_score', '<f8')]),
 (589, 1026, 589, 1., 1.))

In [4]:
if(os.path.exists(f"{database}/halo_dict.pickle")):
    halos = pklload(f"{database}/halo_dict.pickle")
else:
    halos = {'catalog':{}, 'index':{}}
    uri.timer.verbose=0
    for iout in tqdm(np.unique(tree['timestep'])):
        isnap = snaps.get_snap(iout)
        ihals = uhmi.HaloMaker.load(isnap, galaxy=False, double_precision=dp)
        indicies = np.zeros(len(ihals), dtype=int)
        iids = tree[tree['timestep'] == iout]['id']
        ihals = ihals[iids-1]
        indicies[iids-1] = np.arange(len(iids))
        halos['catalog'][iout] = ihals
        halos['index'][iout] = indicies   
    pklsave(halos, f"{database}/halo_dict.pickle")

In [5]:
def read_halo(iout, iid, halos=None):
    i = halos['index'][iout][iid-1]
    return halos['catalog'][iout][i]
read_halo(1020, 1, halos=halos)

(353420, 592, 1020, 1, 592, 0, 45, 21711, 0.85097384, 4.62858909e+11, 0.49432707, 0.48607818, 0.47447001, 75.02861941, 83.49368329, 134.07337116, -3.10094319e+11, -5.3608248e+11, 7.95886209e+11, 0.00385547, 0.11305825, 0.04801294, 0.07035723, 42236.15492919, -36300.89495075, 5935.25997844, 0.03919084, 78.32725047, 0.00138886, 3.71653421e+11, 341822.48651226, 97.57826257, 62.50632897, 0.00138886, 1.79596942e+09)

At 1026

In [6]:
dtype = allsubs.dtype
# ('nparts', '<i4'), ('id', '<i4'), ('timestep', '<i4'), 
# ('level', '<i4'), ('host', '<i4'), ('hostsub', '<i4'), 
# ('nbsub', '<i4'), ('nextsub', '<i4'), ('aexp', '<f8'), 
# ('m', '<f8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), 
# ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('Lx', '<f8'), ('Ly', '<f8'), ('Lz', '<f8'), 
# ('r', '<f8'), ('a', '<f8'), ('b', '<f8'), ('c', '<f8'), 
# ('ek', '<f8'), ('ep', '<f8'), ('et', '<f8'), ('spin', '<f8'), ('sigma', '<f8'), 
# ('rvir', '<f8'), ('mvir', '<f8'), ('tvir', '<f8'), ('cvel', '<f8'), ('rho0', '<f8'), ('rc', '<f8'), 
# ('mcontam', '<f8'), 
# -------------------------------------------------
# ('r10_mem', '<f8'), ('r50_mem', '<f8'), ('r90_mem', '<f8'), 
# ('r10_vir', '<f8'), ('r50_vir', '<f8'), ('r90_vir', '<f8'), 
# ('r10_max', '<f8'), ('r50_max', '<f8'), ('r90_max', '<f8'), 
# ('sub', '<i8'), ('dink', '?'), 
# ('mdm', '<f8'), ('mstar', '<f8'), ('mcold', '<f8'), ('mcell', '<f8'), 
# ('Host', '<i4'), ('r200kpc', '<f8'), ('m200', '<f8'), ('r200', '<f8'), 
# ('mdm_vir', '<f8'), ('mstar_vir', '<f8'), ('mcell_vir', '<f8'), ('mcold_vir', '<f8')]

fname = f"{database}/main_prog/subhalos_01026.pickle"
if(os.path.exists(fname)):
    newsubs = pklload(fname)
else:
    newsubs = np.zeros(
        len(allsubs), dtype=dtype.descr + [('lastid', '<i4'),('give_score', '<f8'), ('take_score', '<f8')])

    for iname in dtype.names:
        newsubs[iname] = allsubs[iname]
    #--------------------------------------------------------------
    # 'lastid', 'give_score', 'take_score'
    #--------------------------------------------------------------
    newsubs['lastid'] = allsubs['id']
    newsubs['give_score'] = 1
    newsubs['take_score'] = 1


    argsort = np.argsort(newsubs['lastid'])
    newsubs = newsubs[argsort]
    pklsave(newsubs, fname)

Other snapshots

In [7]:
cursor = 0
nsub = len(allsubs)
uri.timer.verbose=0
for iout in np.unique(tree['timestep'])[::-1]:
    if(os.path.exists( f"{database}/main_prog/subhalos_{iout:05d}.pickle"))or(iout%10 != 0):
        cursor += nsub
        continue
    isnap = snaps.get_snap(iout)
    ihals = halos['catalog'][iout]
    indicies = halos['index'][iout]

    x1 = np.min(ihals['x']-ihals['r']); x2 = np.max(ihals['x']+ihals['r'])
    y1 = np.min(ihals['y']-ihals['r']); y2 = np.max(ihals['y']+ihals['r'])
    z1 = np.min(ihals['z']-ihals['r']); z2 = np.max(ihals['z']+ihals['r'])
    isnap.box = np.array([[x1,x2],[y1,y2],[z1,z2]])
    uri.timer.verbose=1
    isnap.get_cell(nthread=40, target_fields=['x','y','z','rho','P'], exact_box=False)
    isnap.get_part(nthread=40, target_fields=['x','y','z','m','epoch','id'], exact_box=False)
    uri.timer.verbose=0

    #--------------------------------------------------------------
    #--------------------------------------------------------------
    newsubs = np.zeros(len(allsubs), dtype=dtype.descr + [('lastid', '<i4'),('give_score', '<f8'), ('take_score', '<f8')])

    #--------------------------------------------------------------
    # 'nparts', 'id', 'timestep',
    # 'level', 'host', 'hostsub', 'nbsub', 'nextsub', 
    # 'aexp', 'm', 
    # 'x', 'y', 'z', 'vx', 'vy', 'vz', 'Lx', 'Ly', 'Lz', 
    # 'r', 'a', 'b', 'c', 'ek', 'ep', 'et', 'spin', 'sigma', 
    # 'rvir', 'mvir', 'tvir', 'cvel', 'rho0', 'rc', 'mcontam' 
    #--------------------------------------------------------------
    for iname in dtype.names:
        if(iname in ihals.dtype.names):
            newsubs[iname] = ihals[iname]

    #--------------------------------------------------------------
    # 'lastid', 'give_score', 'take_score'
    #--------------------------------------------------------------
    itree = tree[cursor:cursor+nsub]
    newsubs['lastid'] = itree['lastid']
    newsubs['give_score'] = itree['give_score']
    newsubs['take_score'] = itree['take_score']
    
    for ith in tqdm( range(nsub), desc=f"IOUT{iout:05d} "):
        timereport = []; ref = time.time()
        ihal = ihals[np.argsort(ihals['m'])][ith]
        #--------------------------------------------------------------
        # 'mdm', 'mstar', 'mcold', 'mcell', 
        # 'mdm_vir', 'mstar_vir', 'mcell_vir', 'mcold_vir'
        #--------------------------------------------------------------
        isnap.set_box_halo(ihal, 1, radius_name='r')
        isnap.get_part(nthread=40, target_fields=['x','y','z','m','epoch','id'])
        timereport.append(("get_part", time.time()-ref)); ref = time.time()
        table = isnap.part['star']
        if(len(table)>0):
            table = cut_sphere(table, ihal['x'], ihal['y'], ihal['z'], ihal['r'])
            if(len(table)>0):
                newsubs['mstar'][ith] = np.sum(table['m','Msol'])
                table_vir = cut_sphere(table, ihal['x'], ihal['y'], ihal['z'], ihal['rvir'])
                if(len(table_vir)>0):
                    newsubs['mstar_vir'][ith] = np.sum(table_vir['m','Msol'])
        timereport.append(("cutstar", time.time()-ref)); ref = time.time()
        table = isnap.get_cell(nthread=40, target_fields=['x','y','z','rho','P'])#,'level'])
        timereport.append(("get_cell", time.time()-ref)); ref = time.time()
        table = cut_sphere(table, ihal['x'], ihal['y'], ihal['z'], ihal['r'])
        if(len(table)>0):
            newsubs['mcell'][ith] = np.sum(table['m','Msol'])
            ctable = table[table['T','K']<2e4]
            if(len(ctable)>0):
                newsubs['mcold'][ith] = np.sum(ctable['m','Msol'])
            table_vir = cut_sphere(table, ihal['x'], ihal['y'], ihal['z'], ihal['rvir'])
            if(len(table_vir)>0):
                newsubs['mcell_vir'][ith] = np.sum(table_vir['m','Msol'])
                ctable_vir = table_vir[table_vir['T','K']<2e4]
                if(len(ctable_vir)>0):
                    newsubs['mcold_vir'][ith] = np.sum(ctable_vir['m','Msol'])
        timereport.append(("cut_cell", time.time()-ref)); ref = time.time()
        table = isnap.part['dm']
        table = cut_sphere(table, ihal['x'], ihal['y'], ihal['z'], ihal['r'])
        newsubs['mdm'][ith] = np.sum(table['m','Msol'])
        table_vir = cut_sphere(table, ihal['x'], ihal['y'], ihal['z'], ihal['rvir'])
        newsubs['mdm_vir'][ith] = np.sum(table_vir['m','Msol'])
        timereport.append(("cut_dm", time.time()-ref)); ref = time.time()

        #--------------------------------------------------------------
        # 'r10_mem', 'r50_mem', 'r90_mem', 'r10_vir', 'r50_vir', 'r90_vir', 'r10_max', 'r50_max', 'r90_max', 
        #--------------------------------------------------------------
        all_dist = distance(ihal, table); argsort = np.argsort(all_dist)
        all_dist = all_dist[argsort]; all_mass = table['m'][argsort]
        memdm = uhmi.HaloMaker.read_member_part(isnap, ihal['id'], galaxy=False, target_fields=['x','y','z','m'])
        mem_dist = distance(ihal, memdm); argsort = np.argsort(mem_dist)
        mem_dist = mem_dist[argsort]; mem_mass = memdm['m'][argsort]

        newsubs['r10_mem'][ith] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.1)
        newsubs['r50_mem'][ith] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.5)
        newsubs['r90_mem'][ith] = calc_rhalf_sorted(mem_dist, mem_mass, ratio=0.9)
        newsubs['r10_max'][ith] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.1)
        newsubs['r50_max'][ith] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.5)
        newsubs['r90_max'][ith] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.9)
        _, ind = cut_sphere(table, ihal['x'], ihal['y'], ihal['z'], ihal['rvir'], return_index=True)
        all_dist = all_dist[ind]; all_mass = table['m'][ind]
        newsubs['r10_vir'][ith] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.1)
        newsubs['r50_vir'][ith] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.5)
        newsubs['r90_vir'][ith] = calc_rhalf_sorted(all_dist, all_mass, ratio=0.9)
        timereport.append(("rhalfs", time.time()-ref)); ref = time.time()

        for ireport in timereport:
            print(f"[{ireport[0]}] {ireport[1]:.2f} sec")
    isnap.clear()
    README = "`sub`, `dink`, `Host`, `r200kpc`, `m200`, `r200` are missed!"
    pklsave((newsubs, README), f"{database}/main_prog/subhalos_{iout:05d}.pickle")

    cursor += nsub
    stop()

Reading 4195 AMR & hydro files (78.52 GiB) in /storage6/NewHorizon/snapshots/output_01025... 


Reading cells:   0%|          | 0/4195 [00:00<?, ?it/s]

Done (27.055s).
Reading 4195 part files (31.11 GiB) in /storage6/NewHorizon/snapshots/output_01025... 


Reading parts:   0%|          | 0/4195 [00:00<?, ?it/s]

Done (48.971s).


...

IOUT01025 :   1%|          | 22/3851 [00:05<11:42,  5.45it/s]

[get_part] 0.07 sec
[cutstar] 0.00 sec
[get_cell] 0.15 sec
[cut_cell] 0.00 sec
[cut_dm] 0.00 sec
[rhalfs] 0.00 sec
[get_part] 0.04 sec
[cutstar] 0.00 sec
[get_cell] 0.04 sec
[cut_cell] 0.00 sec
[cut_dm] 0.00 sec
[rhalfs] 0.04 sec
[Signal2] Clearing memory
  ['psm_714fd47c', 'psm_b7645b5a']
	Unlink `psm_b7645b5a`
	Unlink `psm_714fd47c`


IOUT01025 :   1%|          | 22/3851 [00:10<29:22,  2.17it/s]


SystemExit: 0

/home/jeon/.conda/envs/jeonpy/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3406: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
isnap.clear()

98